In [1]:
# ✅ Verificación de versión de Python
import sys

required: tuple[int, int, int] = (3, 10, 0)
current: tuple[int, int, int] = sys.version_info[:3]

assert current >= required, (
    f"⚠️  Se requiere Python {required[0]}.{required[1]}.{required[2]} o superior "
    f"para usar sintaxis moderna de tipos (str | None). "
    f"Detectado: {current[0]}.{current[1]}.{current[2]}"
)

print(f"✅ Python {current[0]}.{current[1]}.{current[2]} >= {required[0]}.{required[1]}.{required[2]}")
print("✅ Sintaxis moderna de tipos disponible (PEP 604: Union | None)")


✅ Python 3.13.3 >= 3.10.0
✅ Sintaxis moderna de tipos disponible (PEP 604: Union | None)


### 🧠 IA + Oracle: Hola mundo agentic
Este notebook guía a los alumnos en una práctica básica para conectar con una base de datos Oracle, generar consultas SQL desde lenguaje natural usando distintos agentes vía [AISuite](https://github.com/andrewyng/aisuite/), y ejecutar esas consultas de forma segura.

El objetivo es que a partir de una consulta en lenguaje natural, un primer agente genere el código sql asociado a esa consulta del usuario. Después se consultará la base de datos y se obtendrá el resultado en formato tabular que se mostrará al usuario. Esa información se enviará a otro agente que interpretará el resultado y se lo explicará al usuario. 

En el fichero Readme.md se explica la configuración necesaria para que este jupiter notebook funcione correctamente.

### 🔄 Flujo del Sistema Agéntico

![Diagrama del Sistema Agéntico](images/diagramaagentic1-notrans.png)

Este diagrama muestra el flujo completo del sistema de agentes que conecta la inteligencia artificial con la Base de Datos Oracle para procesar consultas en lenguaje natural.

El primer paso será cargar las variables de entorno que están guardadas en el fichero .env donde se debe guardar la información de:
- Conexión a la base de datos Oracle ORACLE_HOST, ORACLE_PORT, ORACLE_SID, ORACLE_USER, ORACLE_PASSWORD
- Dependiendo de la IA que se vaya a utilizar se necesitará:
    - OpenAI- OPENAI_API_KEY
    - Anthropic - ANTHROPIC_API_KEY
    - Google - GOOGLE_PROJECT_ID, GOOGLE_REGION, GOOGLE_APPLICATION_CREDENTIALS


### 📚 Sobre las Buenas Prácticas de Código

Este notebook aplica **código profesional Python 3.10+** con:
- ✅ Type hints modernos (`str | None`, `list[tuple]`, `dict[str, str]`)
- ✅ Funciones limpias: UN solo `return`, guard clauses, funciones auxiliares privadas (`_`)
- ✅ Manejo robusto: cierre de recursos, excepciones específicas, validaciones
- ✅ Documentación completa: docstrings con Args, Returns, Raises

**📖 Para detalles completos sobre buenas prácticas de Python (con ejemplos ❌ MAL / ✅ BIEN), consulta:**
👉 **[`buenaspracticas.ipynb`](buenaspracticas.ipynb)**

**💡 Objetivo de este notebook:** Demostrar integración de IA (AISuite) con Oracle para generar SQL desde lenguaje natural.


In [2]:
# 🔐 Cargar variables de entorno y configuración
from dotenv import load_dotenv
import os

def cargar_configuracion() -> dict[str, str | None]:
    """Carga las variables de entorno desde .env.
    
    Returns:
        Diccionario con las variables de configuración
        
    Note:
        Algunas variables pueden ser None si no están definidas
    """
    load_dotenv()
    print("✅ Variables de entorno cargadas.")
    
    config: dict[str, str | None] = {
        "ORACLE_HOST": os.getenv("ORACLE_HOST"),
        "ORACLE_PORT": os.getenv("ORACLE_PORT"),
        "ORACLE_SID": os.getenv("ORACLE_SID"),
        "ORACLE_USER": os.getenv("ORACLE_USER"),
        "ORACLE_PASSWORD": os.getenv("ORACLE_PASSWORD"),
        "GOOGLE_PROJECT_ID": os.getenv("GOOGLE_PROJECT_ID"),
        "GOOGLE_REGION": os.getenv("GOOGLE_REGION"),
        "GOOGLE_APPLICATION_CREDENTIALS": os.getenv("GOOGLE_APPLICATION_CREDENTIALS"),
    }
    
    return config

# Cargar configuración
CONFIG: dict[str, str | None] = cargar_configuracion()


✅ Variables de entorno cargadas.


In [3]:
# 🔗 Conexión a Oracle con Type Hints y Manejo de Recursos
import oracledb

def conectar_oracle(config: dict[str, str | None]) -> tuple[oracledb.Connection, oracledb.Cursor]:
    """Establece conexión a Oracle y retorna conexión y cursor.
    
    Args:
        config: Diccionario con variables de configuración
        
    Returns:
        Tupla con (conexión, cursor) de Oracle
        
    Raises:
        ValueError: Si faltan variables de configuración
        oracledb.DatabaseError: Si falla la conexión
    """
    # Guard clauses: validar configuración
    host: str | None = config.get("ORACLE_HOST")
    port: str | None = config.get("ORACLE_PORT")
    sid: str | None = config.get("ORACLE_SID")
    user: str | None = config.get("ORACLE_USER")
    password: str | None = config.get("ORACLE_PASSWORD")
    
    if not all([host, port, sid, user, password]):
        raise ValueError("Faltan variables de configuración de Oracle")
    
    # Crear DSN y conectar (ya validado, pero mypy no lo sabe)
    dsn: str = oracledb.makedsn(host, port, sid=sid)  # type: ignore
    connection: oracledb.Connection = oracledb.connect(
        user=user,  # type: ignore
        password=password,  # type: ignore
        dsn=dsn
    )
    cursor: oracledb.Cursor = connection.cursor()
    
    # Verificar conexión
    cursor.execute("SELECT 'Conexión exitosa' FROM Dual")
    resultado: tuple = cursor.fetchone()  # type: ignore
    print(f"✅ {resultado[0]}")
    
    return connection, cursor

# Establecer conexión
connection: oracledb.Connection
cursor: oracledb.Cursor
connection, cursor = conectar_oracle(CONFIG)


✅ Conexión exitosa


La estructura de base de datos de ejemplo es muy sencilla. Se muestra el diagrama entidad relación.
![Modelo ER](images/ERBDPEDIDOS.png)

In [4]:
# 🧠 Entrada en lenguaje natural
# Ejemplos para la base de datos de clientes, productos y pedidos

# Seleccionar UNA pregunta (comentar las demás)
pregunta: str = "¿Cuántos pedidos se hicieron en octubre?"
# pregunta = "Selecciona todos los pedidos"
# pregunta = "Cuantos y qué pedidos hizo Ana Gamez"
# pregunta = "Quiero saber los productos que ha comprado Ana Gamez y lo que le costó cada uno"

print(f"📝 Pregunta del usuario: {pregunta}")


📝 Pregunta del usuario: ¿Cuántos pedidos se hicieron en octubre?


Es importante pasarle a los agentes la estructura de la base de datos. Para que las consultas que generen tengan sentido. Eso se hace en el siguiente bloque usando un sistema de descubrimiento.

In [5]:
# 🗂️ Obtener esquema de la base de datos

def obtener_esquema_bd(cursor: oracledb.Cursor, usuario: str | None) -> str:
    """Obtiene el esquema de la base de datos del usuario actual.
    
    Args:
        cursor: Cursor activo de Oracle
        usuario: Nombre del usuario de BD
        
    Returns:
        String formateado con el esquema legible para LLM
        
    Raises:
        ValueError: Si el usuario es None
        oracledb.DatabaseError: Si falla la consulta
    """
    # Guard clause
    if not usuario:
        raise ValueError("Usuario de BD no definido")
    
    # Consultar esquema
    cursor.execute(f"""
        SELECT table_name, column_name
        FROM all_tab_columns
        WHERE owner = UPPER('{usuario}')
        ORDER BY table_name, column_id
    """)
    
    # Construir diccionario de esquema
    esquema_dict: dict[str, list[str]] = {}
    for table, column in cursor:
        esquema_dict.setdefault(table, []).append(column)
    
    # Formatear para LLM (legible y conciso)
    esquema_str: str = "\n".join([
        f"- {tabla}({', '.join(columnas).lower()})" 
        for tabla, columnas in esquema_dict.items()
    ])
    
    print("✅ Esquema de la base de datos obtenido:")
    print(esquema_str)
    
    return esquema_str

# Obtener esquema
esquema: str = obtener_esquema_bd(cursor, CONFIG.get("ORACLE_USER"))


✅ Esquema de la base de datos obtenido:
- APLICACIONES(nombre_aplic, compania, precio, web_aplicacion)
- CLIENTES(id_cliente, nombre, email, ciudad)
- COMPATIBLE(nombre_documento, directorio, nombre_aplicacion)
- DOC_TEXTO(ascii, num_lineas, num_caracteres, tipo, nombre, directorio)
- DOCUMENTO(nombre, directorio, fecha_creacion, comentario, tamano)
- PAGINA(nombre, directorio, numero_pagina)
- PEDIDOS(id_pedido, id_cliente, id_producto, fecha_pedido, cantidad)
- PRODUCTOS(id_producto, nombre, precio, categoria)


In [6]:
# 🤖 Generación de SQL con IA (Gemini|OpenAI|Anthropic vía AISuite)
import aisuite as ai

def generar_sql_desde_lenguaje_natural(
    pregunta: str, 
    esquema: str, 
    modelo: str = "google:gemini-2.0-flash-001"
) -> str:
    """Genera consulta SQL desde lenguaje natural usando IA.
    
    Args:
        pregunta: Consulta del usuario en lenguaje natural
        esquema: Esquema de la BD en formato string
        modelo: Modelo de IA a usar (formato: "proveedor:modelo")
        
    Returns:
        Consulta SQL limpia y lista para ejecutar
    """
    client: ai.Client = ai.Client()
    
    messages: list[dict[str, str]] = [
        {
            "role": "system", 
            "content": f"Eres un asistente experto en SQL para Oracle. Genera solo la consulta SQL compatible con Oracle, sin texto explicativo ni formato Markdown. Usa este esquema:\n{esquema}"
        },
        {"role": "user", "content": pregunta}
    ]
    
    response = client.chat.completions.create(
        model=modelo,
        messages=messages
    )
    
    # Limpiar respuesta
    sql_limpio: str = _limpiar_sql(response.choices[0].message.content.strip())
    
    print(f"✅ SQL generado por {modelo}:")
    print(sql_limpio)
    
    return sql_limpio


def _limpiar_sql(raw_sql: str) -> str:
    """Función auxiliar: Limpia el SQL generado por IA.
    
    Args:
        raw_sql: SQL crudo que puede contener Markdown
        
    Returns:
        SQL limpio en una línea, sin punto y coma
    """
    cleaned: str = raw_sql
    
    # Eliminar bloques de código Markdown
    if cleaned.startswith("```"):
        cleaned = "\n".join(cleaned.split("\n")[1:-1]).strip()
    
    # Normalizar: eliminar punto y coma, saltos de línea, tabulaciones
    cleaned = cleaned.replace(";", "").replace("\n", " ").replace("\t", " ")
    
    # Eliminar espacios múltiples
    return " ".join(cleaned.split())


# Generar SQL
sql_generado: str = generar_sql_desde_lenguaje_natural(pregunta, esquema)


✅ SQL generado por google:gemini-2.0-flash-001:
SELECT COUNT(*) FROM PEDIDOS WHERE EXTRACT(MONTH FROM FECHA_PEDIDO) = 10


In [7]:
# 🧪 Ejecución del SQL con manejo de errores apropiado

def ejecutar_sql(cursor: oracledb.Cursor, sql: str) -> list[tuple]:
    """Ejecuta una consulta SQL y retorna los resultados.
    
    Args:
        cursor: Cursor activo de Oracle
        sql: Consulta SQL a ejecutar
        
    Returns:
        Lista de tuplas con los resultados
        
    Raises:
        oracledb.DatabaseError: Si hay error en la consulta
    """
    # Guard clause
    if not sql.strip():
        raise ValueError("La consulta SQL está vacía")
    
    try:
        cursor.execute(sql)
        resultados: list[tuple] = cursor.fetchall()
        
        print(f"✅ Consulta ejecutada: {len(resultados)} filas obtenidas")
        for fila in resultados:
            print(fila)
            
        return resultados
        
    except oracledb.DatabaseError as e:
        print(f"❌ Error de base de datos: {e}")
        raise

# Ejecutar SQL generado
resultados: list[tuple] = ejecutar_sql(cursor, sql_generado)


✅ Consulta ejecutada: 1 filas obtenidas
(4,)


In [8]:
# 📊 Visualización en formato tabular
import pandas as pd

def convertir_a_dataframe(
    resultados: list[tuple], 
    cursor: oracledb.Cursor
) -> tuple[pd.DataFrame, str]:
    """Convierte resultados de BD a DataFrame y Markdown.
    
    Args:
        resultados: Lista de tuplas con resultados de consulta
        cursor: Cursor con metadata de las columnas
        
    Returns:
        Tupla con (DataFrame, texto_markdown)
    """
    # Extraer nombres de columnas
    columnas: list[str] = [col[0] for col in cursor.description]  # type: ignore
    
    # Crear DataFrame
    df: pd.DataFrame = pd.DataFrame(resultados, columns=columnas)
    
    # Convertir a Markdown para el agente interpretador
    texto_markdown: str = df.to_markdown(index=False)
    
    print("✅ DataFrame creado con éxito")
    return df, texto_markdown

# Convertir resultados
df: pd.DataFrame
texto_resultado: str
df, texto_resultado = convertir_a_dataframe(resultados, cursor)

# Mostrar primeras filas
df.head()


✅ DataFrame creado con éxito


,COUNT(*)
0,4


Aqui se crea el segundo Agente que puede ser con el mismo o diferente proveedor y será el encargado de coger el resultado de la consulta y analizarlo y explicarselo al usuario.

In [9]:
# 🧠 Interpretación de resultados con IA (Segundo Agente)

def interpretar_resultados(
    pregunta: str,
    resultados_markdown: str,
    esquema: str,
    modelo: str = "openai:gpt-4o"
) -> str:
    """Interpreta los resultados de la consulta en lenguaje natural.
    
    Args:
        pregunta: Pregunta original del usuario
        resultados_markdown: Resultados en formato Markdown
        esquema: Esquema de la BD
        modelo: Modelo de IA a usar
        
    Returns:
        Interpretación en lenguaje natural de los resultados
    """
    client: ai.Client = ai.Client()
    
    messages: list[dict[str, str]] = [
        {
            "role": "system", 
            "content": f"Eres un experto en análisis de datos. Resume e interpreta los resultados de una consulta SQL realizada sobre este esquema: {esquema}"
        },
        {
            "role": "user", 
            "content": f"La consulta fue: {pregunta}\n\nY los resultados fueron:\n\n{resultados_markdown}"
        }
    ]
    
    response = client.chat.completions.create(
        model=modelo,
        messages=messages
    )
    
    respuesta: str = response.choices[0].message.content.strip()
    
    print(f"✅ Interpretación generada por {modelo}:")
    print(respuesta)
    
    return respuesta

# Interpretar resultados
respuesta_final: str = interpretar_resultados(
    pregunta=pregunta,
    resultados_markdown=texto_resultado,
    esquema=esquema
)


✅ Interpretación generada por openai:gpt-4o:
La consulta SQL se diseñó para contar el número de pedidos realizados en el mes de octubre. Al observar el resultado de la consulta, podemos interpretar que, en total, se registraron 4 pedidos en el sistema durante el mes de octubre. Esto nos da una idea del nivel de actividad de compras o transacciones ejecutadas por los clientes en ese período específico. Este dato podría ser utilizado para realizar comparaciones con meses anteriores o posteriores, analizar tendencias, y entender patrones de comportamiento de los clientes en relación con las adquisiciones de productos o servicios ofrecidos. También podría ser relevante para la planificación de inventarios y estrategias de marketing.


### 🧠 Reflexión
- ¿Qué otras preguntas podrías hacer?
- ¿Cómo podrías validar que el SQL generado es seguro?
- ¿Qué diferencias hay entre usar Gemini y otros modelos?

### 🔄 Extensión
- Cambia el modelo a `openai:gpt-4o` o `anthropic:claude-3-sonnet`
- Añade un agente que valide que la consulta generada por el primero es correcta.
- Haz que el agente planifique varios pasos (consulta + visualización)

In [ ]:
# 🧹 Limpieza: Cerrar recursos de BD

def cerrar_conexion(cursor: oracledb.Cursor, connection: oracledb.Connection) -> None:
    """Cierra cursor y conexión de Oracle de forma segura.
    
    Args:
        cursor: Cursor a cerrar
        connection: Conexión a cerrar
        
    Note:
        Esta función garantiza el cierre incluso si alguno ya está cerrado
    """
    try:
        if cursor is not None:
            cursor.close()
            print("✅ Cursor cerrado")
    except Exception as e:
        print(f"⚠️  Error al cerrar cursor: {e}")
    
    try:
        if connection is not None:
            connection.close()
            print("✅ Conexión cerrada")
    except Exception as e:
        print(f"⚠️  Error al cerrar conexión: {e}")

# Cerrar recursos (ejecutar al final)
# cerrar_conexion(cursor, connection)
print("💡 Descomentar la línea anterior para cerrar la conexión cuando termines")
